In [4]:
%run ../utils/dataset_building.py

In [2]:
nb_articles=len(get_all_articles("../data/article_categories_no_unknown_types.tsv"))

	Getting all article ids...


In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
max_length=SentenceTransformer('paraphrase-distilroberta-base-v2').max_seq_length
print(f"max_length: {max_length}")
min_length=20

max_length: 256


In [5]:
import re
regex_multiple_spaces=re.compile(r"\s{2,}")

In [5]:
from collections import Counter
token_count_sum_counter=Counter()
def group_sentences(grouped_sentences,section,token_count_sum,section_grouped_sentences,section_total_token_count_dict):
    global token_count_sum_counter
    token_count_sum_counter[token_count_sum]+=1
    if token_count_sum>min_length :
        grouped_sentence=" ".join(grouped_sentences)
        grouped_sentence=re.sub(regex_multiple_spaces,r" ",grouped_sentence)
        section_grouped_sentences.append({'section':section,'sentence':grouped_sentence,'token_count':token_count_sum})

In [7]:
with open("../data/article_section_sentences_token_count.json", "r") as f_in:
    with open("../data/article_section_grouped_sentences.json", "a+") as f_out:
        for line in tqdm(f_in,total=nb_articles):
            json_line=json.loads(line)
            article_id=json_line['article_id']
            section_sentences=json_line['section_sentences_token_count']
            # key: section, value: list of sentences
            section_sentences_dict={}
            # key: section, value: number of tokens in this section
            section_total_token_count_dict={}
            for section_sentence in section_sentences:
                section=section_sentence['section']
                sentence=section_sentence['sentence']
                token_count=section_sentence['token_count']
                if section not in section_sentences_dict.keys():
                    section_sentences_dict[section]=[]
                    section_total_token_count_dict[section]=0
                section_sentences_dict[section].append({'sentence':sentence,'token_count':token_count})
                section_total_token_count_dict[section]+=token_count
                
            section_grouped_sentences=[]
            for section,sentences_dict in section_sentences_dict.items():
                token_count_sum=0
                grouped_sentences=[]
                for sentence_dict in sentences_dict:
                    sentence=sentence_dict['sentence']
                    token_count=sentence_dict['token_count']
                    # we group sentences until the sum of their tokens counts is below max_length
                    if token_count_sum+token_count<=max_length:
                        token_count_sum+=token_count
                        grouped_sentences.append(sentence)
                    else:
                        # once the token counts have reached the max_length limit, we group the sentences into a single
                        group_sentences(grouped_sentences,section,token_count_sum,section_grouped_sentences,section_total_token_count_dict)
                        grouped_sentences.clear()
                        grouped_sentences.append(sentence)
                        token_count_sum=token_count
                if len(grouped_sentences)>0:
                    group_sentences(grouped_sentences,section,token_count_sum,section_grouped_sentences,section_total_token_count_dict)
            
            if len(section_grouped_sentences)>0:
                f_out.write(json.dumps({'article_id':article_id,'section_grouped_sentences':section_grouped_sentences})+"\n")

  0%|          | 0/2048191 [00:00<?, ?it/s]

In [8]:
# proportion of section contents that were removed because of the min_length
sum([v for k,v in token_count_sum_counter.items() if k <min_length])/sum([v for v in token_count_sum_counter.values()])

0.04590493054744201

In [9]:
# proportion of sentences which were already too long before the grouping,
# those will be truncaned before being passed to the model
sum([v for k,v in token_count_sum_counter.items() if k >max_length])/sum([v for v in token_count_sum_counter.values()])

0.001307963328792129